# import model and datasets

In [1]:
from models.resnet import ResNet34
import torch
from collections import OrderedDict
from torchvision.transforms import transforms
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
# import sklearn.covariance
from torch.autograd import Variable
import random
from FS import FS
device = "cuda:0"

model = ResNet34(10).to(device)
fault_model = ResNet34(10).to(device)
checkpoint = torch.load("./pre_trained/resnet34_cifar10.pth")
new_state_dict = OrderedDict()
for k,v in checkpoint.items():
    new_key = k.replace('module.','')
    new_state_dict[new_key] = v
model.load_state_dict(new_state_dict)


"""
ID data -> Cifar 10
Profiling(mean and covariance is done on Cifar 10)
"""
data_path = './data'
transform_test = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize using CIFAR-10 mean and std
])
test_dataset = torchvision.datasets.CIFAR10(
    root='./data',  # Path to the dataset folder
    train=False,    # Indicate this is the test dataset
    download=True,  # Download the dataset if not available
    transform=transform_test  # Apply the transformations
)

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',  # Path to the dataset folder
    train=True,    # Indicate this is the test dataset
    download=True,  # Download the dataset if not available
    transform=transform_test  # Apply the transformations
)
# Create the DataLoader for the test dataset
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=1,   # Set the batch size (you can adjust this)
    shuffle=False,   # Do not shuffle for testing
    num_workers=2    # Number of subprocesses to use for data loading
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=1,   # Set the batch size (you can adjust this)
    shuffle=False,   # Do not shuffle for testing
    num_workers=2    # Number of subprocesses to use for data loading
)

# Create test datasets
svhn_test_dataset = torchvision.datasets.SVHN(root=data_path, split='test', download=True, transform=transform_test)
isun_test_dataset = torchvision.datasets.ImageFolder(root=f"{data_path}/isun", transform=transform_test)
lsun_test_dataset = torchvision.datasets.ImageFolder(root=f"{data_path}/lsun_resize", transform=transform_test)

# Create test data loaders
batch_size = 1  # Set your batch size here
svhn_test_loader = DataLoader(svhn_test_dataset, batch_size=batch_size, shuffle=False)
isun_test_loader = DataLoader(isun_test_dataset, batch_size=batch_size, shuffle=False)
lsun_test_loader = DataLoader(lsun_test_dataset, batch_size=batch_size, shuffle=False)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
autoAttack_data = torch.load('./adv_samples/densenet3_cifar10_autoattack.pt')
data_list, label_list = zip(*autoAttack_data)
inputs_tensor = torch.stack([torch.from_numpy(data).float() for data in data_list])
labels_tensor = torch.tensor(label_list, dtype=torch.long)

AA_dataset = TensorDataset(inputs_tensor, labels_tensor)
Autoattack_test_loader = DataLoader(AA_dataset, batch_size=1, shuffle=False)
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
fgsm_data = torch.load('./adv_samples/densenet3_cifar10_fgsm.pt')
data_list, label_list = zip(*fgsm_data)
inputs_tensor = torch.stack([torch.from_numpy(data).float() for data in data_list])
labels_tensor = torch.tensor(label_list, dtype=torch.long)
fsgm_dataset = TensorDataset(inputs_tensor, labels_tensor)
fgsm_test_loader = DataLoader(fsgm_dataset, batch_size=1, shuffle=False)
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
pgd_data = torch.load('./adv_samples/densenet3_cifar10_pgd.pt')
data_list, label_list = zip(*pgd_data)
inputs_tensor = torch.stack([torch.from_numpy(data).float() for data in data_list])
labels_tensor = torch.tensor(label_list, dtype=torch.long)
pgd_dataset = TensorDataset(inputs_tensor, labels_tensor)
pgd_test_loader = DataLoader(pgd_dataset, batch_size=1, shuffle=False)


/tmp/ipykernel_1322333/666494735.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./pre_trained/resnet34_cifar10.pth")


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/test_32x32.mat


/tmp/ipykernel_1322333/666494735.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoAttack_data = torch.load('./adv_samples/densenet3_cifar10_autoattack.pt')
/tmp/ipyk

In [2]:
cohort_size = 32
activations = {}

# Define the hook function
def get_activation(name):
    def hook(module, input, output):
        activations[name] = output.detach()
    return hook

for name, module in model.named_modules():
    module.register_forward_hook(get_activation(name))


model.eval()
count = 0
with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.cuda()
        layer1_features = model(inputs)
        count +=1 
        if count > 0:
            break



# function to get score from activations

In [ ]:
import math 
def get_score(Activation, mini, maxi, means,covs):
    
    for item in Activation.keys():
        num = len((Activation[item]))
        break

    score = {}
    normalized_score ={}
    mini_score  = {}
    sum_score = {}
    
    
    final_sum_Score = {}
    final_min_score = {}
    count = {}
    for j in range(num):
        #  for every image j
        total_min =1
        # total_sum =1
        layer_Sum_score  = {}
        layer_min_score  = {}
        final_sum_Score[j] = []
        final_min_score[j] = []
        for neuron in Activation:
            score[neuron] = []
            normalized_score[neuron] =[]
            val_ID = Activation[neuron][j]
            count[neuron] =0 
            for i in range(1,10):   
                divi = np.max( [ abs(means[neuron][i]- mini[neuron][i]) , abs(means[neuron][i]- maxi[neuron][i]) ])
                if divi == 0:
                    count[neuron] += 1
                    # print(f' { neuron } with divi = 0 detected ')
                    divi = 1
                # if covs[neuron][i] == 0:
                #     score[neuron] += [1]
                # else :
                score[neuron] += [ abs(means[neuron][i] - val_ID) ]
                normalized_score[neuron] = [ abs(means[neuron][i] - val_ID) / divi]
            sum_score[neuron] = np.sum(normalized_score[neuron])
            mini_score[neuron] = np.min(score[neuron])
            layer, loc=  neuron 
            if layer in layer_min_score :
                layer_Sum_score[layer] += sum_score[neuron]
                layer_min_score[layer] += mini_score[neuron]
            else:
                layer_Sum_score[layer] = sum_score[neuron]
                layer_min_score[layer] = mini_score[neuron]

        prev_min_score = 0
        prev_sum_score = 0
        for layer in layer_min_score:
            total_min += layer_min_score[layer]
            total_sum += layer_Sum_score[layer]
            
            final_min_score[j] += [layer_min_score[layer] + prev_min_score]
            final_sum_Score[j] += [layer_Sum_score[layer]+ prev_sum_score]
            # prev_sum_score = total_sum
            # prev_min_score = total_min

        
        # final_min_score += [total_min]
        # final_sum_Score += [total_sum]  
        if j%100 == 0:
            print(j)
    
    return final_min_score, final_sum_Score

In [4]:
# select neurons with highest activations

# import torch

# selected_neurons = []
# num_layer = 0

# for layer, act in activations.items():
#     output_dim = act.shape
#     num_layer += 1

#     # Flatten the activation tensor to 1D
#     flat_act = act.view(-1)
    
#     # Determine the number of neurons to select
#     num_to_select = min(cohort_size, flat_act.numel())
    
#     if num_to_select == 0:
#         print(f"{layer} = layer, number of neurons = 0")
#         continue
    
#     # Get the top 'num_to_select' activation values and their indices
#     top_values, top_indices = torch.topk(flat_act, num_to_select, largest=True, sorted=True)
    
#     # Convert flat indices back to multi-dimensional positions
#     for idx in top_indices:
#         # Make sure idx is a tensor and call unravel_index
#         pos = torch.unravel_index(idx, output_dim)
#         neuron = (layer, tuple(pos))
#         selected_neurons.append(neuron)
    
#     print(f"{layer} = layer, number of neurons = {num_to_select}")
# print(num_layer)
# print(f"Total selected neurons: {len(selected_neurons)}")



# Choose random neurons in a model 

In [5]:
selected_neurons=[]
num_layer = 0

for layer, act in activations.items():
    output_dim = list(act.shape)
    num = 0
    flag = 0 
    num_neuron = 0
    num_layer += 1
    while num < cohort_size:
         
        pos = tuple(random.randint(0,output_dim[i]-1) for i in range(len(output_dim)))
        neuron = (layer,pos)
        # print(neuron)
        if flag == 100 :
            break 
        if neuron in selected_neurons:
            flag += 1 
            continue
        selected_neurons.append(neuron)
        num +=1
    print(layer, " = layer, number of neurons = " ,num)
print((selected_neurons))


conv1  = layer, number of neurons =  32
bn1  = layer, number of neurons =  32
layer1.0.conv1  = layer, number of neurons =  32
layer1.0.bn1  = layer, number of neurons =  32
layer1.0.conv2  = layer, number of neurons =  32
layer1.0.bn2  = layer, number of neurons =  32
layer1.0.shortcut  = layer, number of neurons =  32
layer1.0  = layer, number of neurons =  32
layer1.1.conv1  = layer, number of neurons =  32
layer1.1.bn1  = layer, number of neurons =  32
layer1.1.conv2  = layer, number of neurons =  32
layer1.1.bn2  = layer, number of neurons =  32
layer1.1.shortcut  = layer, number of neurons =  32
layer1.1  = layer, number of neurons =  32
layer1.2.conv1  = layer, number of neurons =  32
layer1.2.bn1  = layer, number of neurons =  32
layer1.2.conv2  = layer, number of neurons =  32
layer1.2.bn2  = layer, number of neurons =  32
layer1.2.shortcut  = layer, number of neurons =  32
layer1.2  = layer, number of neurons =  32
layer1  = layer, number of neurons =  32
layer2.0.conv1  = la

In [6]:
layer = "conv1"  # Replace with the actual layer name
layer,pos = selected_neurons[0]  # Get the indices for the neuron
b,c,h,w = pos
# Access the activation tensor for the specified layer
print(pos)
if layer in activations:
    activation_tensor = activations[layer]  # Shape: [batch_size, channels, height, width]
    print(activation_tensor[b,c,h,w])
    # Access the specific neuron's activation
    neuron_activation = activation_tensor[b, c, h, w]
    print(f"Activation for neuron [{b}, {c}, {h}, {w}] in layer '{layer}': {neuron_activation}")
else:
    print(f"Layer '{layer}' not found in activations.")

(0, 0, 3, 6)
tensor(-2.8378e-07, device='cuda:0')
Activation for neuron [0, 0, 3, 6] in layer 'conv1': -2.8377547778291046e-07


# Collect activations for training data 

all_neuron_activations[layer][neuron][label] = value

In [ ]:
def extract_activations(loader, model, selected_neurons, num):
    all_neural_activations = {}
    features = []
    labels = []
    model.eval()
    accuracy = []
    with torch.no_grad():
        
        count = 0
        for inputs, targets in loader:
            targets = targets.item()
            count +=1 
            inputs = inputs.cuda()
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            if predicted != targets:
                accuracy += [count]
            for neuron in selected_neurons:
                layer,pos = neuron
                if len(pos) > 2:       
                    b,c,w,h=pos
                    value = activations[layer][b,c,w,h].item()
                    if neuron not in all_neural_activations:
                        all_neural_activations[neuron]= {}
                    if targets not in all_neural_activations[neuron]:
                        all_neural_activations[neuron][targets] = []    
                    all_neural_activations[neuron][targets] += [value]
                elif len(pos) ==2 :       
                    b,c=pos
                    value = activations[layer][b,c].item()
                    if neuron not in all_neural_activations:
                        all_neural_activations[neuron]= {}
                    if targets not in all_neural_activations[neuron]:
                        all_neural_activations[neuron][targets] = []    
                    all_neural_activations[neuron][targets]+= [value]
                else :
                    print(" locha in number of parameters in neurons")
            if count % 100 == 0:
                print(count)
            if count > num:
                break
        print(accuracy , " =  error in fault free model  ")
    return all_neural_activations ,accuracy 


In [8]:
Id_activations, Wrong = extract_activations(train_loader,model,selected_neurons,2000)
#

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
[]  =  error in fault free model  


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Create dictionaries to store statistics (optional)
means = {}
covs = {}
maxi = {}
mini = {}


neuron_list = list(selected_neurons)  # Assuming the same neurons are present in both datasets
layers= [ ]
 
for neuron in neuron_list:
    layer,_ = neuron
    
    means[neuron] = {}
    covs[neuron] = {}
    mini[neuron] = {}
    maxi[neuron] = {}
    

    id_list = [Id_activations[neuron][i] for i in range(10)]

    for i, (id_lst) in enumerate(zip(id_list), 1):
        # Calculate statistics for Id_activations
        means[neuron][i] = np.mean(id_lst)
        covs[neuron][i] = np.cov(id_lst)
        mini[neuron][i] = np.min(id_lst)
        maxi[neuron][i] = np.max(id_lst)

    

# extract activations during inference

In [15]:

def extract_activations_inference(loader, model, selected_neurons, num):
    all_neural_activations = {}
    accuracy = []
    model.eval()
    with torch.no_grad():
        
        count = 0
        for inputs, targets in loader:
            targets = targets.item()
            
            inputs = inputs.cuda()
            outputs =model(inputs)
            _, predicted = outputs.max(1)
            accuracy += [predicted] 
            for neuron in selected_neurons:
                layer,pos = neuron
                if count ==  0:
                    all_neural_activations[neuron]= []
                if len(pos) > 2:       
                    b,c,w,h=pos
                    value = activations[layer][b,c,w,h].item()
                    all_neural_activations[neuron] += [value]
                elif len(pos) ==2 :       
                    b,c=pos
                    value = activations[layer][b,c].item()    
                    all_neural_activations[neuron] += [value]
                
            if count % 100 == 0:
                print(count)
            if count > num:
                break
            count +=1 
    return all_neural_activations ,accuracy


def extract_faulty_activations_inference(loader, model, selected_neurons, num,fs):
    all_neural_activations = {}

    model.eval()
    accuracy = []
    fs.setLayerInfo(model)
    with torch.no_grad():

        count = 0
        for inputs, targets in loader:
            targets = targets.item()
            
            inputs = inputs.cuda()
            handle = fs.onlineSingleLayerOutputInjection(model=model, targetLayer='random', NofError=1, targetBit='random')
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            accuracy += [predicted] 
            handle.remove()
            for neuron in selected_neurons:
                layer,pos = neuron
                if count ==  0:
                    all_neural_activations[neuron]= []
                if len(pos) > 2:       
                    b,c,w,h=pos
                    value = activations[layer][b,c,w,h].item()
                    all_neural_activations[neuron] += [value]
                elif len(pos) ==2 :       
                    b,c=pos
                    value = activations[layer][b,c].item()    
                    all_neural_activations[neuron] += [value]
                
            if count % 100 == 0:
                print(count)
            if count > num:
                break
            count +=1 
    return all_neural_activations, accuracy

In [14]:
print(selected_neurons[0])

('conv1', (0, 0, 3, 6))


In [16]:
ID_test_neural_activation , Wrong_ID = extract_activations_inference(test_loader,model,selected_neurons,10000)
print("id done")
fs = FS()
SDC_neural_Activations , wrong_SDC= extract_faulty_activations_inference(test_loader,model,selected_neurons,10000,fs)
print(' faulty done')


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
id done
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
 faulty done


In [17]:
ID_min_score, ID_sum_score = get_score(ID_test_neural_activation,mini,maxi,means,covs)
print(" Id score done")
SDC_min_score, SDC_sum_score = get_score(SDC_neural_Activations,mini,maxi,means,covs)
print( " sdc score done")

0


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
 Id score done
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
 sdc score 

In [18]:
SDC = []
for val in range(len(wrong_SDC)):
    if wrong_SDC[val]!= Wrong_ID[val]:
        SDC += [val]

In [80]:
len(SDC)

93

In [ ]:
# """
# scores for all the neurons for  Id data and 1 OOD data (SVHN)
# """
# import pandas as pd
# scores_ID = {}
# scores_SVHN ={}
# df = pd.DataFrame()
# sum_ID_score = {}
# mini_ID_score = {}
# sum_SVHN_score = {}
# mini_SVHN_score = {}

# Final_Sum_score_Id = {}
# Final_min_score_Id = {}
# Final_Sum_score_SVHN = {}
# Final_min_score_SVHN = {}

# for neuron in ID_test_neural_activation:
#     # print(neuron)
#     # print(ID_test_neural_activation[neuron])
#     scores_ID[neuron] = []
#     scores_SVHN[neuron] = []
#     val_ID = ID_test_neural_activation[neuron][0]
#     val_SVHN = Svhn_neural_activation[neuron][0]
#     for i in range(1,10):   
#         # print(means[neuron][i] , )
#         # print(val_ID)
#         # print(mini[neuron][i])
#         # print(maxi[neuron][i])
#         # ab = abs(means[neuron][i] - val_ID)
#         # bc = np.max( [ abs(means[neuron][i]- mini[neuron][i]) , abs(means[neuron][i]- maxi[neuron][i]) ])
#         # print(ab," divide by ", bc)
#         scores_ID[neuron] += [ abs(means[neuron][i] - val_ID)/ np.max( [ abs(means[neuron][i]- mini[neuron][i]) , abs(means[neuron][i]- maxi[neuron][i]) ]) ]
#         scores_SVHN[neuron] += [ abs(means[neuron][i] - val_SVHN)/np.max( [ abs(means[neuron][i]- mini[neuron][i]) , abs(means[neuron][i]- maxi[neuron][i]) ]) ]
       

#     sum_ID_score[neuron] = np.sum(scores_ID[neuron])
#     mini_ID_score[neuron] = np.min(scores_ID[neuron])

#     sum_SVHN_score[neuron] = np.sum(scores_SVHN[neuron])
#     mini_SVHN_score[neuron] = np.min(scores_SVHN[neuron])
#     # sum_Svhn = np.sum(scores_SVHN[neuron])
#     layer, loc=  neuron 
#     # Final_Sum_score_Id[layer] = 0
#     # Final_min_score_Id[layer] = 0
#     # Final_Sum_score_SVHN[layer] = 0
#     # Final_min_score_SVHN[layer] = 0
#     if layer in Final_min_score_Id :
#         Final_Sum_score_Id[layer] += sum_ID_score[neuron]
#         Final_min_score_Id[layer] += mini_ID_score[neuron]
#         Final_Sum_score_SVHN[layer] += sum_SVHN_score[neuron]
#         Final_min_score_SVHN[layer] += mini_SVHN_score[neuron]
#     else:
#         Final_Sum_score_Id[layer] = 0
#         Final_min_score_Id[layer] = 0
#         Final_Sum_score_SVHN[layer] = 0
#         Final_min_score_SVHN[layer] = 0


# for layer in Final_min_score_Id:
#     total_score_ID
#     print(Final_min_score_Id[layer] , " min score ------",Final_min_score_SVHN[layer] )
# for layer in Final_min_score_Id:  
#     print(Final_Sum_score_Id[layer] , " sum score" , Final_Sum_score_SVHN[layer]) 


In [94]:
ID_min_avg = []
ID_sum_avg = []
for val in range(500):
    ID_min_avg  +=  [np.sum(ID_min_score[val])]
    ID_sum_avg += [np.sum(ID_sum_score[val])]

print(ID_min_avg)
print(ID_sum_avg)
ID_min_avg = np.sum(ID_min_avg)/500
ID_sum_avg = np.sum(ID_sum_avg)/500
print(ID_min_avg)
print(ID_sum_avg)

[455.71416822827763, 424.09613509358815, 392.5010950916063, 389.2589802928466, 429.2230472654497, 383.59665614843414, 444.0695220032606, 369.17024364438447, 384.91504899213606, 399.1674332579891, 353.8387536858737, 502.3844449694572, 431.57347473090215, 476.68122325190194, 486.1807916937154, 429.60649752305045, 471.4677024779303, 451.56565337345313, 388.17839944370695, 413.55361093995464, 407.2589689943237, 375.2441030618154, 357.8573972667211, 504.1544787693421, 402.41605343007217, 429.0754013857872, 399.85473186168826, 391.1741503082897, 464.1662263137957, 458.60573091756, 427.3934668632576, 348.9065958108642, 373.11932512375114, 414.849557413295, 410.7381484499629, 418.50496722788876, 413.54571402145496, 446.25965122885003, 470.3780939733304, 405.2305786123291, 452.5322805117682, 433.46400101673123, 443.5780848537563, 383.27915222674187, 408.952101300858, 437.66701407603773, 374.1312628191474, 475.5809133187907, 384.9891142208007, 350.2848857805341, 472.6348396297137, 393.4871351534

In [ ]:
df = [1,2,3,4,4,5,6,7]
df= df[-2:]
print(df)

[6, 7]


In [102]:
SDC_min = []
SDC_sum =[]
for val in SDC_min_score:
    r_min = abs(np.sum(SDC_min_score[val][:])/ID_min_avg -1)
    r_sum = abs(np.sum(SDC_sum_score[val][:])/ ID_sum_avg -1)
    if r_min > 1:
        SDC_min += [val+1]
    if r_sum > 1:
        SDC_sum += [val+1]

print(SDC)
print(len(SDC_min))
print(len(SDC_sum))

    
False_SDC_min = []
False_SDC_sum =[]
for val in range(9500):
    r_min = abs(np.sum(ID_min_score[val+500])/ID_min_avg -1)
    r_sum = abs(np.sum(ID_sum_score[val+500])/ ID_sum_avg -1)
    if r_min > 0.3:
        False_SDC_min += [val+1]
    if r_sum > 0.3:
        False_SDC_sum += [val+1]

print(SDC)
print(len(False_SDC_min))
print(len(False_SDC_sum))


[19, 217, 307, 433, 493, 520, 537, 708, 801, 911, 991, 1195, 1255, 1275, 1398, 1827, 2076, 2400, 2531, 2608, 2803, 2807, 2903, 3107, 3145, 3204, 3244, 3250, 3273, 3279, 3285, 3428, 3504, 3736, 3876, 4168, 4229, 4326, 4564, 4626, 4657, 4762, 4863, 4882, 5173, 5224, 5649, 5730, 5856, 5931, 5993, 6185, 6193, 6387, 6458, 6770, 6900, 6939, 7227, 7278, 7352, 7461, 7550, 7591, 7592, 7902, 7967, 8009, 8025, 8329, 8334, 8375, 8390, 8655, 8938, 9026, 9070, 9102, 9103, 9162, 9184, 9246, 9249, 9281, 9334, 9428, 9533, 9584, 9606, 9716, 9793, 9936, 9975]
116
116
[19, 217, 307, 433, 493, 520, 537, 708, 801, 911, 991, 1195, 1255, 1275, 1398, 1827, 2076, 2400, 2531, 2608, 2803, 2807, 2903, 3107, 3145, 3204, 3244, 3250, 3273, 3279, 3285, 3428, 3504, 3736, 3876, 4168, 4229, 4326, 4564, 4626, 4657, 4762, 4863, 4882, 5173, 5224, 5649, 5730, 5856, 5931, 5993, 6185, 6193, 6387, 6458, 6770, 6900, 6939, 7227, 7278, 7352, 7461, 7550, 7591, 7592, 7902, 7967, 8009, 8025, 8329, 8334, 8375, 8390, 8655, 8938, 9026, 

# Print the score

In [104]:
import csv 

false_positive_min = 0
false_positive_sum = 0
for val in SDC_min:
    if val not in SDC:
        false_positive_min += 1
for val in SDC_sum:
    if val not in SDC:
        false_positive_sum += 1
detect_SDC_minScore = 0
detect_SDC_sumScore = 0
for val in SDC:
    if val in SDC_min:
        detect_SDC_minScore += 1
    if val in SDC_sum:
        detect_SDC_sumScore += 1

header = ['Number of samples','Total Sdc',' Total SDC Detected by min_Score', 'False Positives min_score', 'SDC Detected by sum_score', 'False positive sum_score' ]
row= [10000,len(SDC),detect_SDC_minScore,false_positive_min,detect_SDC_sumScore,false_positive_sum]
filename = 'SDC_Results.csv'
with open(filename, mode='w', newline='',) as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(header)
    writer.writerow(row)

[603, 802]
[48, 122, 140, 387, 434, 544, 553, 588, 662, 940, 951]
11
2


dict_values([10.27164434895911, 10.691337769379484, 6.884534334823391, 7.4713813039873855, 6.520854216029641, 6.749248780977476, 2.530323759924983, 3.5786568927344775, 6.612837689048612, 6.5258341745727515, 7.690054953518077, 6.376500711456061, 3.3964643708299684, 4.550151711055306, 7.381529962548958, 7.604593596784481, 7.182595078170464, 6.522049148041811, 4.471773284283823, 5.767009222136739, 6.005872452741041, 8.157875827411658, 8.27134381124117, 8.00853764817488, 7.739363530579573, 7.609501956978524, 7.917087270449778, 7.226696985383511, 4.681851474608056, 8.839708291687456, 9.002594141493558, 8.073090747122352, 5.923633812655252, 4.992303176734657, 4.40197558310415, 8.69168213735051, 8.373468232078006, 6.51793085215224, 6.118364414167944, 4.481584101640679, 5.635755213894753, 8.418837424262234, 8.27748809723956, 7.161230960544229, 5.456251755058022, 4.45946502702579, 4.0684871505901095, 4.711950165038221, 8.62097467598224, 7.753824759759993, 8.996194938572028, 9.1026848101671, 7.9

In [ ]:

for i in SDC:
    if i not in SDC_min:
        print(i," -> SDC not detected?")


In [ ]:
for i in range(100):
    print(ID_sum_score[i])

In [ ]:
for i in range(100):
    print(SDC_sum_score[i])